# 内容ブーステッド協調フィルタリング

In [35]:
import numpy as np
import pandas as pd

In [36]:
rating = [
    { "user_id": 1, "item_id": 2, "rating": 4},
    { "user_id": 1, "item_id": 3, "rating": 3},
    { "user_id": 1, "item_id": 4, "rating": 1},
    { "user_id": 1, "item_id": 5, "rating": 2},
    { "user_id": 1, "item_id": 7, "rating": 4},
    { "user_id": 2, "item_id": 1, "rating": 5},
    { "user_id": 2, "item_id": 2, "rating": 5},
    { "user_id": 2, "item_id": 3, "rating": 4},
    { "user_id": 2, "item_id": 5, "rating": 3},
    { "user_id": 2, "item_id": 6, "rating": 3},
    { "user_id": 2, "item_id": 7, "rating": 5},
    { "user_id": 2, "item_id": 8, "rating": 4},
    { "user_id": 3, "item_id": 1, "rating": 4},
    { "user_id": 3, "item_id": 3, "rating": 5},
    { "user_id": 3, "item_id": 4, "rating": 3},
    { "user_id": 3, "item_id": 5, "rating": 2},
    { "user_id": 3, "item_id": 8, "rating": 3},
    { "user_id": 4, "item_id": 2, "rating": 3},
    { "user_id": 4, "item_id": 4, "rating": 2},
    { "user_id": 4, "item_id": 5, "rating": 1},
    { "user_id": 4, "item_id": 6, "rating": 1},
    { "user_id": 4, "item_id": 7, "rating": 3},
    { "user_id": 5, "item_id": 1, "rating": 2},
    { "user_id": 5, "item_id": 2, "rating": 1},
    { "user_id": 5, "item_id": 3, "rating": 2},
    { "user_id": 5, "item_id": 4, "rating": 4},
    { "user_id": 5, "item_id": 6, "rating": 3},
    { "user_id": 5, "item_id": 7, "rating": 2},
]

In [37]:
rating_df = pd.DataFrame(rating)
rating_df.groupby("item_id")["rating"].describe()

,count,mean,std,min,25%,50%,75%,max
item_id,,,,,,,,
1,3.0,3.666667,1.527525,2.0,3.00,4.0,4.50,5.0
2,4.0,3.250000,1.707825,1.0,2.50,3.5,4.25,5.0
3,4.0,3.500000,1.290994,2.0,2.75,3.5,4.25,5.0
4,4.0,2.500000,1.290994,1.0,1.75,2.5,3.25,4.0
5,4.0,2.000000,0.816497,1.0,1.75,2.0,2.25,3.0
6,3.0,2.333333,1.154701,1.0,2.00,3.0,3.00,3.0
7,4.0,3.500000,1.290994,2.0,2.75,3.5,4.25,5.0
8,2.0,3.500000,0.707107,3.0,3.25,3.5,3.75,4.0


In [38]:
item = [
    {"item_id": 1, "type": "赤身"},
    {"item_id": 2, "type": "赤身"},
    {"item_id": 3, "type": "赤身"},
    {"item_id": 4, "type": "白身"},
    {"item_id": 5, "type": "白身"},
    {"item_id": 6, "type": "白身"},
    {"item_id": 7, "type": "光物"},
    {"item_id": 8, "type": "光物"},
    {"item_id": 9, "type": "光物"},
]
item_df = pd.DataFrame(item)
item_df.head()

,item_id,type
0,1,赤身
1,2,赤身
2,3,赤身
3,4,白身
4,5,白身


In [39]:
user_item = rating_df.merge(item_df, on="item_id", how="right")
train = user_item.copy()
matrix = user_item.pivot(index="user_id", columns="item_id", values="rating")
matrix.drop(matrix.index[0], inplace=True)
matrix

item_id,1,2,3,4,5,6,7,8,9
user_id,,,,,,,,,
1.0,NaN,4.0,3.0,1.0,2.0,NaN,4.0,NaN,NaN
2.0,5.0,5.0,4.0,NaN,3.0,3.0,5.0,4.0,NaN
3.0,4.0,NaN,5.0,3.0,2.0,NaN,NaN,3.0,NaN
4.0,NaN,3.0,NaN,2.0,1.0,1.0,3.0,NaN,NaN
5.0,2.0,1.0,2.0,4.0,NaN,3.0,2.0,NaN,NaN


In [40]:
user_item.head()

,user_id,item_id,rating,type
0,1.0,2,4.0,赤身
1,2.0,2,5.0,赤身
2,4.0,2,3.0,赤身
3,5.0,2,1.0,赤身
4,1.0,3,3.0,赤身


In [41]:
Ui = rating_df.groupby('user_id').agg({"item_id": list})["item_id"].to_dict()

In [42]:
train.drop(train.index[28], inplace=True)

In [48]:
# 評価したtypeの平均値で欠損値を埋める
for user_id in matrix.index:
    for item_id in matrix.columns:
        if ~np.isnan(matrix.loc[user_id, item_id]):
            continue
        groupby = train.groupby(["user_id", "type"])["rating"].mean()
        item_type = item_df[item_df["item_id"]==item_id]["type"]
        matrix.loc[user_id, item_id] = groupby[user_id][item_type].iloc[-1]

In [49]:
matrix

item_id,1,2,3,4,5,6,7,8,9
user_id,,,,,,,,,
1.0,3.5,4.0,3.0,1.0,2.0,1.5,4.0,4.0,4.0
2.0,5.0,5.0,4.0,3.0,3.0,3.0,5.0,4.0,4.5
3.0,4.0,4.5,5.0,3.0,2.0,2.5,3.0,3.0,3.0
4.0,3.0,3.0,3.0,2.0,1.0,1.0,3.0,3.0,3.0
5.0,2.0,1.0,2.0,4.0,3.5,3.0,2.0,2.0,2.0


In [97]:
# ユーザ同士の類似度計算
def sim(u, v):
    u_diff = u - np.mean(u)
    v_diff = v - np.mean(v)
    numerator = np.dot(u_diff, v_diff)
    denominator = np.sqrt(np.sum(u_diff**2))*np.sqrt(np.sum(v_diff**2))
    
    return round(numerator / denominator, 3)

In [98]:
S = np.zeros((len(matrix.index), len(matrix.index)))
user_id2index = dict(zip(matrix.index, range(len(matrix.index))))
for i in matrix.index:
    u_index = user_id2index[i]
    for j in matrix.index:
        v_index = user_id2index[j]
        S[u_index, v_index] = sim(matrix.loc[i,:].values, matrix.loc[j,:].values)

In [99]:
# 相関行列　S
S

array([[ 1.   ,  0.887,  0.404,  0.828, -0.899],
       [ 0.887,  1.   ,  0.561,  0.848, -0.87 ],
       [ 0.404,  0.561,  1.   ,  0.683, -0.649],
       [ 0.828,  0.848,  0.683,  1.   , -0.772],
       [-0.899, -0.87 , -0.649, -0.772,  1.   ]])

In [105]:
# 信頼度に応じて行列Sに重みを付与
alpha = 6
w1 = len(Ui[1]) / alpha
w2 = 1

w_h12 = (2*w1*w2)/(w1+w2)
w_h12

0.9090909090909091

In [106]:
w_s12 = len(np.intersect1d(Ui[1], Ui[2])) / alpha
w_s12

0.6666666666666666

In [ ]:
w_c12 = (w_h12 + w_)